# WhatsApp Business Cloud API

#### Imports

In [7]:
import requests, json, pandas as pd

# Get quote from quotable api

In [10]:
# api_url = r'https://zenquotes.io/api/'  # api base url
api_url = r'https://api.quotable.io/'
### Get random quote
params = {
#     'key': api_key,
    'minLenght': 1,
    'maxLenght': 5000,
    'tags': 'famous-quotes|wisdom|friendship|inspirational|success',
#     'author': None,
#     'authorId': None,
}

endpoint = 'random?' # random, quotes, tags, authors, search/quotes, search/authors
url = api_url + endpoint # + urllib.parse.urlencode(params)
payload, headers = {}, {}
response = requests.request("GET", url, params=params)

In [11]:
res = json.loads(response.text); quote = res; res

{'_id': 'OATcVT3HPV',
 'content': 'Wisdom ceases to be wisdom when it becomes too proud to weep, too grave to laugh, and too selfish to seek other than itself.',
 'author': 'Kahlil Gibran',
 'tags': ['wisdom'],
 'authorSlug': 'kahlil-gibran',
 'length': 124,
 'dateAdded': '2021-05-07',
 'dateModified': '2021-05-07'}

In [12]:
### Get authors
endpoint = 'authors'
url = api_url + endpoint # urllib.parse.urlencode(params)
response = requests.request("GET", url, params={'limit': 1000, 'page': 1,'slug': res['authorSlug']})
res = json.loads(response.text)

In [15]:
author = res['results'][0]
pd.DataFrame(res['results']).nlargest(5, 'quoteCount')

,_id,name,link,bio,description,quoteCount,slug,dateAdded,dateModified
0,h05GYnKzckM3,Kahlil Gibran,https://en.wikipedia.org/wiki/Kahlil_Gibran,"Gibran Khalil Gibran (January 6, 1883 – April ...",Lebanese artist and author,14,kahlil-gibran,2020-01-27,2020-01-27


In [14]:
res

{'count': 1,
 'totalCount': 1,
 'page': 1,
 'totalPages': 1,
 'lastItemIndex': None,
 'results': [{'_id': 'h05GYnKzckM3',
   'name': 'Kahlil Gibran',
   'link': 'https://en.wikipedia.org/wiki/Kahlil_Gibran',
   'bio': 'Gibran Khalil Gibran (January 6, 1883 – April 10, 1931), was a Lebanese-American writer, poet and visual artist, also considered a philosopher although he himself rejected this title.',
   'description': 'Lebanese artist and author',
   'quoteCount': 14,
   'slug': 'kahlil-gibran',
   'dateAdded': '2020-01-27',
   'dateModified': '2020-01-27'}]}

In [13]:
### Extra: Translate response
import translators as ts

def translate(query):
    return ts.google(query_text=query, from_language='en', to_language='pt')

translate(quote['content'])

'A sabedoria deixa de ser sabedoria quando fica orgulhoso demais para chorar, grave demais para rir e egoísta demais para procurar além de si mesmo.'

---

# Send messages

#### Credentials

In [3]:
credentials = json.loads(open('credentials.json', 'r').read())

token = credentials['token']
number_id = credentials['number_id']
namespace = credentials['template_namespace']

#### Api address & phone message endpoint

In [4]:
WhatsAppCloudAPIUrl = 'https://graph.facebook.com/v13.0'
msg_url = WhatsAppCloudAPIUrl + f'/{number_id}/messages'; msg_url # messages endpoint

'https://graph.facebook.com/v13.0/101349609340848/messages'

### Send text message (user-initiated conversation only)

In [36]:
payload = {
  "messaging_product": "whatsapp",
  "recipient_type": "individual",
  "to": "5531984258358",
  "type": "text",
  "text": json.dumps({
    "preview_url": "False",
    "body": "MESSAGE_CONTENT"
  })
}

In [37]:
params, headers = {}, {'Authorization': f'Bearer {token}'}
response = requests.request("POST", msg_url, headers=headers, data=payload, params=params)
res = json.loads(response.text); res

{'messaging_product': 'whatsapp',
 'contacts': [{'input': '5531984258358', 'wa_id': '553184258358'}],
 'messages': [{'id': 'wamid.HBgMNTUzMTg0MjU4MzU4FQIAERgSRUNFMTZFNjQxOEU5Njk1RkUyAA=='}]}

In [50]:
params, headers = {}, {}
response = requests.request("POST", msg_url, headers=headers, data=payload, params=params)
res = json.loads(response.text); res

{'error': {'message': "Unsupported post request. Object with ID '101349609340848' does not exist, cannot be loaded due to missing permissions, or does not support this operation. Please read the Graph API documentation at https://developers.facebook.com/docs/graph-api",
  'type': 'GraphMethodException',
  'code': 100,
  'error_subcode': 33,
  'fbtrace_id': 'A2kuD44-6MFqClRy2G5lL-F'}}

---

### Send template message (user or business-initiated conversation)

#### Template:

    Frases para te motivar todos os dias.

    *{{1}}*

    Autor: {{2}}

    Tags: {{3}}

#### Sample message:

    Frases para te motivar todos os dias.
    
    Quando o destino nos entrega um limão, vamos tentar fazer limonada.

    Autor: Dale Carnegie

    Tags: Citações famosas

#### Formatting functions

In [16]:
def translate(query):
    if type(query)==str:
        return ts.google(query_text=query.replace('-', ' '), from_language='en', to_language='pt')
    elif type(query)==list:
        return list(map(translate, query))

# Extra: Capitalize single string or list
def capitalize(query):
    if type(query)==str:
        return query.capitalize()
    elif type(query)==list:
        return list(map(capitalize, query))

as_text_obj = lambda text: {'type': 'text', 'text': text}

def as_text_object_list(text_list):
    return list(map(as_text_obj, text_list))

#### Define post request params

In [35]:
text_list = [
    translate(quote['content']), quote['author'],
    translate(author['description']), author['link'],
    ', '.join(capitalize(translate(quote['tags']))) + '.'
]

In [39]:
payload = {
  "messaging_product": "whatsapp",
  "recipient_type": "individual",
  "to": "5531984258358", # try multiple divided by ','.
  "type": "template",
  "template": json.dumps({
#     "namespace": credentials['template_namespace'], # template namespace id
    "name": "citacao_famosa", # template name id
    "language": {
      "code": "pt_BR"
    },
    "components": [
      {
        "type": "body",
        "parameters": as_text_object_list(text_list)
      }
    ]
  })
}

#### Send request with message params to api url

In [40]:
params, headers = {}, {'Authorization': f'Bearer {token}'}
response = requests.request("POST", msg_url, headers=headers, data=payload, params=params)
res = json.loads(response.text); res

{'messaging_product': 'whatsapp',
 'contacts': [{'input': '5531984258358', 'wa_id': '553184258358'}],
 'messages': [{'id': 'wamid.HBgMNTUzMTg0MjU4MzU4FQIAERgSNUZFREEwOTFGRDFBQTAzMTc4AA=='}]}

Obs: Correct response output for successful message.

# Send text message using BotCity WhatsApp Plugin

#### Install WhatsApp Cloud API BotCity Plugin

In [2]:
!pip install botcity-whatsapp-plugin

  Created wheel for botcity-whatsapp-plugin: filename=botcity_whatsapp_plugin-0.1.0-py3-none-any.whl size=5654 sha256=2c12aa10896ed69b8eb2c3c080c78a8d5675316d52cece08301f9eab0cc58d1b
  Stored in directory: c:\users\luisr\appdata\local\pip\cache\wheels\1e\3b\a0\a373c38c0c6e13f3a6f9ea5ee5bc926262e0c17dc34cccf846
Successfully built botcity-whatsapp-plugin


#### Import and instantiate plugin

In [9]:
from botcity.plugins.whatsapp import BotWhatsappPlugin
# Instantiate the plugin
whatsapp = BotWhatsappPlugin(access_token=token, whatsapp_number_id=number_id)

### Sending test message

In [18]:
# Phone numbers must be used in format:
# [country code][phone number including area code]
whatsapp.send_text_message("5531984258358", "Hey! Sending a simple message! 😃")